In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Data/p7_master_window.csv')
df.fillna(value=0, inplace=True)
df = df[['op_num_people', 'of_pose_distance', 'of_gaze_distance', 'of_success', 'of_confidence', 'ros_difficulty', 'ros_mistakes_session', 'ros_ts_robot_talked', 'engagement']]

len_df = len(df)
split_ratio = 0.8
split = int(split_ratio * len_df)

train_df, test_df = df.iloc[:split,:], df.iloc[split:,:]

feats = []
for i in df.columns:
    if i!= 'engagement':
        feats.append(i)
label = ['engagement']

X = df[feats]
Y = df[label]
X_train = train_df[feats]
y_train = train_df[label]
X_test = test_df[feats]
y_test = test_df[label]

In [3]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16100, 8) (16100, 1)
(4026, 8) (4026, 1)


In [4]:
X_train = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape, y_train.shape)

(16100, 8, 1) (16100, 1)


In [5]:
# create the model
model = Sequential()
model.add(LSTM(100, input_shape=(8,1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train.values, validation_data=(X_test, y_test.values), epochs=10, batch_size=32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 16100 samples, validate on 4026 samples
Epoch 1/10
16100/16100 [==============================] - 4s 256us/step - loss: 0.4084 - acc: 0.8142 - val_loss: 0.3378 - val_acc: 0.8425
Epoch 2/10
16100/16100 [==============================] - 4s 240us/step - loss: 0.3592 - acc: 0.8468 - val_loss: 0.3168 - val_acc: 0.8753
Epoch 3/10
16100/16100 [==============================] - 3s 185us/step - loss: 0.3522 - acc: 0.8475 - val_loss: 0.2996 - val_acc: 0.8808
Epoch 4/10
16100/16100 [==============================] - 3s 184us/step - loss: 0.3475 - acc: 0.8512 - val_loss: 0.2931 - val_acc: 0.8843
Epoch 5/10
16100/16100 [==============================] - 3s 185us/step - loss: 0.3414 - acc: 0.8520 - val_loss: 0.3037 - val_acc: 0.8783
Epoch 6/10
16100/16100 [==

In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.77%


In [8]:
model.metrics_names

['loss', 'acc']